In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install EMD-signal

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.7 MB/s eta 0:00:00


In [ ]:
import matplotlib.pyplot as plt
import time
import scipy
import scipy.io
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
from torch.utils import data
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import pandas as pd
from PyEMD import EMD
from PyEMD import EEMD

In [ ]:
import sys    
sys_path = '/content/drive/MyDrive/EMG-classification'
sys.path.append(sys_path)
from util import *
from dataloader import *
from model import *

In [ ]:
# DataLoader

record_data0501 = np.load('/content/drive/MyDrive/EMG-classification/dataset/rawdata/record_data0501.npy')
record_data0502 = np.load('/content/drive/MyDrive/EMG-classification/dataset/rawdata/record_data0502.npy')
record_data0503 = np.load('/content/drive/MyDrive/EMG-classification/dataset/rawdata/record_data0503.npy')
record_data0505 = np.load('/content/drive/MyDrive/EMG-classification/dataset/rawdata/record_data0505.npy')
record_data0506 = np.load('/content/drive/MyDrive/EMG-classification/dataset/rawdata/record_data0506.npy')
labels0501 = np.load('/content/drive/MyDrive/EMG-classification/dataset/label/label0501.npy')
labels0502 = np.load('/content/drive/MyDrive/EMG-classification/dataset/label/label0502.npy')
labels0503 = np.load('/content/drive/MyDrive/EMG-classification/dataset/label/label0503.npy')
labels0505 = np.load('/content/drive/MyDrive/EMG-classification/dataset/label/label0505.npy')
labels0506 = np.load('/content/drive/MyDrive/EMG-classification/dataset/label/label0506.npy')

signal0501 = record_data0501.flatten()
signal0502 = record_data0502.flatten()
signal0503 = record_data0503.flatten()
signal0505 = record_data0505.flatten()
signal0506 = record_data0506.flatten()

assert (len(signal0501) == record_data0501.shape[0] * record_data0501.shape[1])
assert (len(signal0502) == record_data0502.shape[0] * record_data0502.shape[1])
assert (len(signal0503) == record_data0503.shape[0] * record_data0503.shape[1])
assert (len(signal0505) == record_data0505.shape[0] * record_data0505.shape[1])
assert (len(signal0506) == record_data0506.shape[0] * record_data0506.shape[1])
print("label shape:{}\nrecord_data.shape:{}\n".format(labels0501.shape, record_data0501.shape))
assert (len(labels0501) == record_data0501.shape[0])

label shape:(5746, 1)
record_data.shape:(5746, 1200)



In [ ]:
def save_model(path, epoch, model, optimizer):
    ckpt = {
    "epoch" : epoch,
    "model_state_dict" : model.state_dict(),
    "optimizer_state_dict" : optimizer.state_dict(),
    # "scheduler_state_dict" : scheduler.state_dict(),
    }
    print("saving model to:", path)
    torch.save(ckpt,path)
# path = '/content/drive/MyDrive/EMG-classification/logs/eemd0129/test_eemd_ep' + str(epoch) + ".pt"
# save_model(path, epoch, model, optimizer)

train_label = np.concatenate((labels0502, labels0503, labels0505, labels0506), axis=0)
train_record_data = np.concatenate((record_data0502, record_data0503, record_data0505, record_data0506), axis=0)
train_signal = np.concatenate((signal0502, signal0503, signal0505, signal0506))
assert train_label.shape[0] == train_record_data.shape[0]
print("train data:{}\ntest data:{}\n".format(train_record_data.shape, record_data0501.shape))
val_record_data, val_signal, val_label = record_data0501, signal0501, labels0501

eemd = EEMD()
emd = eemd.EMD
emd.extrema_detection="parabol"
fs = np.linspace(0,1,1200)

train data:(25229, 1200)
test data:(5746, 1200)



In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int,float)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

In [ ]:
# input is added with window
eps = 1
input_size = 129
#set hidden size
hidden_size = 256
n_class = 9
learning_rate = 0.01
max_epochs = 20
middle_features = 128
model = Netone(num_classes=n_class, hidden_size=input_size, num_layers=1, middle_feature=middle_features) #num_classes, hidden_size, num_layers, bidirectional, middle_feature
#model.cuda()
# criterion = nn.CrossEntropyLoss()
criterion = FocalLoss()
optimizer = torch.optim.Adam(model.parameters())

train_acc = []
val_acc = []

for epoch in range(max_epochs):
    correct_all = 0
    total_loss = 0
    train_length = 0
    predicted_train_csv = []
    real_train_csv = []

    for batch_idx in range(0,train_record_data.shape[0],eps):
        # tic = time.perf_counter()
        if batch_idx > train_record_data.shape[0]-eps: break
        train_eIMFs = train_signal[1200*batch_idx:1200*(batch_idx+eps)+1]
        train_raw = spectro(train_record_data[batch_idx:batch_idx+eps], train_eIMFs, 100, 256)
        train_raw = torch.from_numpy(train_raw)
        label_raw = train_label[batch_idx:batch_idx+eps]
        # print("train_raw shape:{}\n label_raw shape:{}".format(train_raw.shape, label_raw.shape)) # (1, 129, 56)

        label = np.repeat(label_raw, train_raw.shape[-1])
        label = torch.from_numpy(label)
        train_set = train_raw.permute(0, 2, 1)
        train_set = train_set.reshape(train_set.shape[0]*train_set.shape[1], -1)
        # print("train_set shape:{}\nlabel shape:{}\n".format(train_set.shape, label.shape)) # [310284, 129]

        batch_x = torch.unsqueeze(train_set, dim=0) # [1, 263736, 129]
        batch_x = Variable(batch_x.float())  # .cuda()
        batch_y = label.long()

        optimizer.zero_grad()
        output = model(batch_x)  # ,hidden = model(batch_x,None)

        # toc = time.perf_counter()
        # print(f"test epochs time in {toc - tic:0.4f} seconds")

        _, predicted_train = torch.max(output.data, 1)
        assert(len(predicted_train)==len(batch_y))
        correct_all += sum(batch_y.eq(predicted_train))
        train_length += len(batch_y)

        predicted_train_csv.append(predicted_train.cpu().detach().numpy())
        real_train_csv.append(batch_y.cpu().detach().numpy())

        loss = criterion(output, batch_y.squeeze())#.float().reshape(-1, 1)
        total_loss += loss
        loss.backward()
        optimizer.step()

    np.save('/content/drive/MyDrive/EMG-classification/logs/eemd_focal/val01_train_pred_'+ str(correct_all / train_length) + '.npy', predicted_train_csv)
    np.save('/content/drive/MyDrive/EMG-classification/logs/eemd_focal/val01_train_real_'+ str(correct_all / train_length) + '.npy', real_train_csv)

    print('\n************************************************************************************************')
    print("Epoch {}/{}:, Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch+1,
        max_epochs,
        float(total_loss / train_length),
        float(optimizer.param_groups[0]['lr'])))
    print('train_acc:{}\n'.format(correct_all/train_length))

    if epoch%2==0:
        correct_all = 0
        predicted_val_csv = []
        real_val_csv = []
        val_length = 0
        for batch_idx in range(0,val_record_data.shape[0],eps):
            # tic = time.perf_counter()
            if batch_idx > val_record_data.shape[0]-eps: break
            val_eIMFs = eemd.eemd(val_signal[1200*batch_idx:1200*(batch_idx+eps)+1])
            val_raw = spectro(val_record_data[batch_idx:batch_idx+eps], val_eIMFs[0], 100, 256)
            val_raw = torch.from_numpy(val_raw)
            label_raw = val_label[batch_idx:batch_idx+eps]
            # print("val_raw shape:{}\n label_raw shape:{}".format(val_raw.shape, label_raw.shape)) # (1, 129, 56)

            label = np.repeat(label_raw, val_raw.shape[-1])
            label = torch.from_numpy(label)
            val_set = val_raw.permute(0, 2, 1)
            val_set = val_set.reshape(val_set.shape[0]*val_set.shape[1], -1)
            # print("val_set shape:{}\nlabel shape:{}\n".format(val_set.shape, label.shape)) # [310284, 129]

            batch_x = torch.unsqueeze(val_set, dim=0) # [1, 263736, 129]
            batch_x = Variable(batch_x.float())  # .cuda()
            batch_y = label.long()

            output = model(batch_x)  # ,hidden = model(batch_x,None)

            # toc = time.perf_counter()
            # print(f"Test time cost in {toc - tic:0.4f} seconds")

            _, predicted_val = torch.max(output.data, 1)
            # print("predicted_val shape:{}\nbatch_y shape:{}\n".format(predicted_val.shape, batch_y.shape))
            # if len(predicted_val)!=len(batch_y): 
            #   print("label shape:{}\nval_raw shape:{}\nlabel_raw shape:{}".format(label.shape, val_raw.shape, label_raw.shape))
            correct_all += sum(batch_y.eq(predicted_val))
            val_length += len(batch_y)
            predicted_val_csv.append(predicted_val.cpu().detach().numpy())
            real_val_csv.append(batch_y.cpu().detach().numpy())

        print('\n************************************************************************************************')
        print("Epoch {}/{}:\ntest_acc:{}\n".format(epoch+1, max_epochs, correct_all/val_length))

        np.save('/content/drive/MyDrive/EMG-classification/logs/eemd_focal/val01_val_pred_'+ str(correct_all / val_length) + '.npy', predicted_val_csv)
        np.save('/content/drive/MyDrive/EMG-classification/logs/eemd_focal/val01_val_real_'+ str(correct_all / val_length) + '.npy', real_val_csv)

<ipython-input-7-341bab4101f7>:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)



************************************************************************************************
Epoch 1/20:, Train Loss 0.0084, Learning Rate 0.0010
train_acc:0.8487532734870911


************************************************************************************************
Epoch 1/20:
test_acc:0.569265604019165


************************************************************************************************
Epoch 2/20:, Train Loss 0.0130, Learning Rate 0.0010
train_acc:0.7378385663032532


************************************************************************************************
Epoch 3/20:, Train Loss 0.0139, Learning Rate 0.0010
train_acc:0.7407851219177246


************************************************************************************************
Epoch 3/20:
test_acc:0.569265604019165


************************************************************************************************
Epoch 4/20:, Train Loss 0.0170, Learning Rate 0.0010
train_acc:0.6940397620201111

